In [3]:
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

True

In [4]:
from typing import Annotated

import mermaid
from langchain_core.runnables.config import RunnableConfig
from langchain_openai import ChatOpenAI
from langchain_teddynote import logging
from langchain_teddynote.tools.tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from typing_extensions import TypedDict


# 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]


logging.langsmith("CH17-LangGraph")

# 메모리 저장소 생성
memory = MemorySaver()

# 도구 초기화
tool = TavilySearch(max_results=3)
tools = [tool]

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

# 챗봇 노드 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 상태 그래프 초기화
graph_builder = StateGraph(State)

# 노드 추가
graph_builder.add_node("chatbot", chatbot)

# 도구 노드 생성
tool_node = ToolNode(tools=[tool])

# 그래프에 도구 노드 추가
graph_builder.add_node("tools", tool_node)

# 조건부 엣지
graph_builder.add_conditional_edges("chatbot",tools_condition)

# tools > chatbot
graph_builder.add_edge("tools", "chatbot")

# START > chatbot
graph_builder.add_edge(START, "chatbot")

# chatbot > END
graph_builder.add_edge("chatbot", END)

# 그래프 컴파일
graph = graph_builder.compile(checkpointer=memory)

# 그래프 시각화
mermaid_str = graph.get_graph().draw_mermaid()
mermaid.Mermaid(mermaid_str).to_png("output.png")

config = RunnableConfig(
        recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
        configurable={"thread_id": "1"},  # 스레드 ID 설정
)

LangSmith 추적을 시작합니다.
[프로젝트명]
CH17-LangGraph


In [5]:
question = (
        "내 이름은 `테디노트` 입니다. YouTube 채널을 운영하고 있어요. 만나서 반가워요"
)

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()

# 이어지는 질문
question = "내 이름이 뭐라고 했지?"

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()

================================== Ai Message ==================================

안녕하세요, 테디노트님! 만나서 반가워요. YouTube 채널에 대해 더 알고 싶어요. 어떤 콘텐츠를 주로 제작하시나요? 도움이 필요하시거나 궁금한 점이 있다면 언제든지 말씀해 주세요!
================================== Ai Message ==================================

귀하의 이름은 "테디노트"입니다.


In [ ]:
from langchain_core.runnables import RunnableConfig

question = "내 이름이 뭐라고 했지?"

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "2"},  # 아까와는 다른 스레드 ID 설정
)

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()

================================== Ai Message ==================================

죄송하지만, 저에게는 사용자 정보를 기억하는 기능이 없어서 당신의 이름을 알 수 없습니다. 당신의 이름을 말씀해 주시면 그에 맞춰 대화할 수 있습니다.


In [7]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)
# 그래프 상태 스냅샷 생성
snapshot = graph.get_state(config)
snapshot.values["messages"]

[HumanMessage(content='내 이름은 `테디노트` 입니다. YouTube 채널을 운영하고 있어요. 만나서 반가워요', additional_kwargs={}, response_metadata={}, id='c23ef69c-43ad-4c31-95d4-1bf4f82f822c'),
 AIMessage(content='안녕하세요, 테디노트님! 만나서 반가워요. YouTube 채널에 대해 더 알고 싶어요. 어떤 콘텐츠를 주로 제작하시나요? 도움이 필요하시거나 궁금한 점이 있다면 언제든지 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 118, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CPqr0p0eHGAWHnTiDD24toqqOfZrT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--11282001-ecbb-4997-83d2-29942a26bebc-0', usage_metadata={'input_tokens': 118, 'output_tokens': 55, 'total_tokens': 173, 'input_token_details': {'audio': 0, 'ca

In [10]:
snapshot.values

{'messages': [HumanMessage(content='내 이름은 `테디노트` 입니다. YouTube 채널을 운영하고 있어요. 만나서 반가워요', additional_kwargs={}, response_metadata={}, id='c23ef69c-43ad-4c31-95d4-1bf4f82f822c'),
  AIMessage(content='안녕하세요, 테디노트님! 만나서 반가워요. YouTube 채널에 대해 더 알고 싶어요. 어떤 콘텐츠를 주로 제작하시나요? 도움이 필요하시거나 궁금한 점이 있다면 언제든지 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 118, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CPqr0p0eHGAWHnTiDD24toqqOfZrT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--11282001-ecbb-4997-83d2-29942a26bebc-0', usage_metadata={'input_tokens': 118, 'output_tokens': 55, 'total_tokens': 173, 'input_token_details': {'

In [8]:
from langchain_teddynote.messages import display_message_tree

# 메타데이터(tree 형태로 출력)
display_message_tree(snapshot.metadata)

    source: "loop"
    step: 4
    parents: {}
